You can download the `requirements.txt` for this course from the workspace of this lab. `File --> Open...`

# L2: Create Agents to Research and Write an Article

In this lesson, you will be introduced to the foundational concepts of multi-agent systems and get an overview of the crewAI framework.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import from the crewAI libray.

In [2]:
from crewai import Agent, Task, Crew

- As a LLM for your agents, you'll be using OpenAI's `gpt-3.5-turbo`.

**Optional Note:** crewAI also allow other popular models to be used as a LLM for your Agents. You can see some of the examples at the [bottom of the notebook](#1).

In [3]:
import os
from utils import get_openai_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'

## Creating Agents

- Define your Agents, and provide them a `role`, `goal` and `backstory`.
- It has been seen that LLMs perform better when they are role playing.

### Agent: Planner

**Note**: The benefit of using _multiple strings_ :
```Python
varname = "line 1 of text"
          "line 2 of text"
```

versus the _triple quote docstring_:
```Python
varname = """line 1 of text
             line 2 of text
          """
```
is that it can avoid adding those whitespaces and newline characters, making it better formatted to be passed to the LLM.

In [4]:
planner = Agent(
    role="Content Planner",
    goal="Plan engaging and factually accurate content on {topic}",
    backstory="You're working on planning a blog article "
              "about the topic: {topic}."
              "You collect information that helps the "
              "audience learn something "
              "and make informed decisions. "
              "Your work is the basis for "
              "the Content Writer to write an article on this topic.",
    allow_delegation=False,
	verbose=True
)

### Agent: Writer

In [5]:
writer = Agent(
    role="Content Writer",
    goal="Write insightful and factually accurate "
         "opinion piece about the topic: {topic}",
    backstory="You're working on a writing "
              "a new opinion piece about the topic: {topic}. "
              "You base your writing on the work of "
              "the Content Planner, who provides an outline "
              "and relevant context about the topic. "
              "You follow the main objectives and "
              "direction of the outline, "
              "as provide by the Content Planner. "
              "You also provide objective and impartial insights "
              "and back them up with information "
              "provide by the Content Planner. "
              "You acknowledge in your opinion piece "
              "when your statements are opinions "
              "as opposed to objective statements.",
    allow_delegation=False,
    verbose=True
)

### Agent: Editor

In [6]:
editor = Agent(
    role="Editor",
    goal="Edit a given blog post to align with "
         "the writing style of the organization. ",
    backstory="You are an editor who receives a blog post "
              "from the Content Writer. "
              "Your goal is to review the blog post "
              "to ensure that it follows journalistic best practices,"
              "provides balanced viewpoints "
              "when providing opinions or assertions, "
              "and also avoids major controversial topics "
              "or opinions when possible.",
    allow_delegation=False,
    verbose=True
)

## Creating Tasks

- Define your Tasks, and provide them a `description`, `expected_output` and `agent`.

### Task: Plan

In [7]:
plan = Task(
    description=(
        "1. Prioritize the latest trends, key players, "
            "and noteworthy news on {topic}.\n"
        "2. Identify the target audience, considering "
            "their interests and pain points.\n"
        "3. Develop a detailed content outline including "
            "an introduction, key points, and a call to action.\n"
        "4. Include SEO keywords and relevant data or sources."
    ),
    expected_output="A comprehensive content plan document "
        "with an outline, audience analysis, "
        "SEO keywords, and resources.",
    agent=planner,
)

### Task: Write

In [8]:
write = Task(
    description=(
        "1. Use the content plan to craft a compelling "
            "blog post on {topic}.\n"
        "2. Incorporate SEO keywords naturally.\n"
		"3. Sections/Subtitles are properly named "
            "in an engaging manner.\n"
        "4. Ensure the post is structured with an "
            "engaging introduction, insightful body, "
            "and a summarizing conclusion.\n"
        "5. Proofread for grammatical errors and "
            "alignment with the brand's voice.\n"
    ),
    expected_output="A well-written blog post "
        "in markdown format, ready for publication, "
        "each section should have 2 or 3 paragraphs.",
    agent=writer,
)

### Task: Edit

In [9]:
edit = Task(
    description=("Proofread the given blog post for "
                 "grammatical errors and "
                 "alignment with the brand's voice."),
    expected_output="A well-written blog post in markdown format, "
                    "ready for publication, "
                    "each section should have 2 or 3 paragraphs.",
    agent=editor
)

## Creating the Crew

- Create your crew of Agents
- Pass the tasks to be performed by those agents.
    - **Note**: *For this simple example*, the tasks will be performed sequentially (i.e they are dependent on each other), so the _order_ of the task in the list _matters_.
- `verbose=2` allows you to see all the logs of the execution. 

In [10]:
crew = Crew(
    agents=[planner, writer, editor],
    tasks=[plan, write, edit],
    verbose=2
)

## Running the Crew

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [11]:
result = crew.kickoff(inputs={"topic": "Artificial Intelligence"})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on Artificial Intelligence.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan: 

Title: The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

Introduction:
- Brief overview of the current landscape of artificial intelligence
- Importance of staying up to date with the latest trends and key players in the industry

Key Points:
1. Latest Trends in Artificial Intelligence:
- Machine learning advancements
- AI in healthcare and finance
- Ethical considerations in AI development

2. Key Players in Artificial Intelligence:
- Google DeepMind
- I

I now can give a great answer

Final Answer:
# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction

Artificial intelligence (AI) is rapidly transforming various aspects of our lives, from healthcare to finance, through continuous advancements in machine learning. Staying abreast of the latest trends and key players in the AI industry is crucial for individuals looking to leverage its power effectively. As AI becomes more integrated into society, understanding its impact and ethical considerations is essential.

## Latest Trends in Artificial Intelligence

A notable trend in AI is the ongoing advancements in machine learning, leading to more sophisticated algorithms that enable AI systems to learn and adapt in real-time. The applications of AI in healthcare and finance are expanding, from personalized medicine to algorithmic trading. However, the ethical development and responsible use of AI are paramount to avoid potential harm as its

- Display the results of your execution as markdown in the notebook.

In [12]:
from IPython.display import Markdown
Markdown(result)

# The Future of Artificial Intelligence: Latest Trends, Key Players, and Noteworthy News

## Introduction

Artificial intelligence (AI) is rapidly transforming various aspects of our lives, from healthcare to finance, through continuous advancements in machine learning. Staying abreast of the latest trends and key players in the AI industry is crucial for individuals looking to leverage its power effectively. As AI becomes more integrated into society, understanding its impact and ethical considerations is essential.

## Latest Trends in Artificial Intelligence

A notable trend in AI is the ongoing advancements in machine learning, leading to more sophisticated algorithms that enable AI systems to learn and adapt in real-time. The applications of AI in healthcare and finance are expanding, from personalized medicine to algorithmic trading. However, the ethical development and responsible use of AI are paramount to avoid potential harm as its presence grows.

## Key Players in Artificial Intelligence

Leading the charge in AI innovation are key players such as Google DeepMind, renowned for its work in deep learning and reinforcement learning. IBM Watson is another major player, providing AI-powered solutions across diverse industries. Tesla Neural Networks, under the guidance of Elon Musk, are pushing the boundaries of AI in autonomous vehicles and other domains.

## Noteworthy News in Artificial Intelligence

Recent breakthroughs in natural language processing, exemplified by models like GPT-3, have impressed the AI community with their language generation capabilities. However, concerns about the impact of AI on the job market are rising as automation threatens traditional roles. Regulatory efforts and privacy concerns surrounding AI technologies are also gaining traction as policymakers grapple with the implications of widespread AI adoption.

## Conclusion

Remaining informed about the latest trends, key players, and noteworthy news in the AI industry is crucial as AI continues to progress. By following reputable AI news sources and considering the ethical dimensions of AI development, we can ensure that AI benefits society as a whole. Whether you are a tech enthusiast, business professional, or student, engagement with the dynamic world of AI is essential for unlocking its full potential.

## Try it Yourself

- Pass in a topic of your choice and see what the agents come up with!

In [13]:
topic = "YOUR TOPIC HERE"
result = crew.kickoff(inputs={"topic": topic})

 [DEBUG]: == Working Agent: Content Planner
 [INFO]: == Starting Task: 1. Prioritize the latest trends, key players, and noteworthy news on YOUR TOPIC HERE.
2. Identify the target audience, considering their interests and pain points.
3. Develop a detailed content outline including an introduction, key points, and a call to action.
4. Include SEO keywords and relevant data or sources.


> Entering new CrewAgentExecutor chain...
I now can give a great answer

Final Answer: 

Content Plan for Blog Article on "Impact of Social Media on Mental Health"

1. Latest Trends, Key Players, and Noteworthy News:
- Latest trend: Increase in social media usage during the pandemic
- Key players: Facebook, Instagram, Twitter, TikTok
- Noteworthy news: Studies linking excessive social media use to mental health issues

2. Target Audience:
- Young adults aged 18-35
- Interested in mental health awareness and social media trends
- Pain points: Feeling overwhelmed by constant comparison on social media, st

I now can give a great answer

Final Answer:
# The Impact of Social Media on Mental Health

In today's fast-paced digital world, social media plays a significant role in shaping how we connect with others and consume information. The prevalence of platforms like Facebook, Instagram, Twitter, and TikTok has undoubtedly revolutionized the way we interact with the world around us. However, along with the benefits of increased connectivity and entertainment, there is a growing concern about the impact of social media on mental health.

Numerous studies have pointed out the negative consequences of excessive social media use, particularly among young adults. The constant exposure to curated images and lifestyles on these platforms can lead to feelings of inadequacy, comparison, and anxiety. In addition, cyberbullying and online harassment have emerged as serious issues that can have detrimental effects on an individual's mental well-being.

Despite the potential drawbacks, social media also

In [14]:
Markdown(result)

# The Impact of Social Media on Mental Health

In today's fast-paced digital world, social media plays a significant role in shaping how we connect with others and consume information. The prevalence of platforms like Facebook, Instagram, Twitter, and TikTok has undoubtedly revolutionized the way we interact with the world around us. However, along with the benefits of increased connectivity and entertainment, there is a growing concern about the impact of social media on mental health.

Numerous studies have pointed out the negative consequences of excessive social media use, particularly among young adults. The constant exposure to curated images and lifestyles on these platforms can lead to feelings of inadequacy, comparison, and anxiety. In addition, cyberbullying and online harassment have emerged as serious issues that can have detrimental effects on an individual's mental well-being.

Despite the potential drawbacks, social media also offers a platform for positive experiences and support. Many individuals find solace in online communities that provide a sense of belonging and understanding. Furthermore, social media serves as a valuable resource for mental health information and support services, empowering users to seek help when needed. 

In order to maintain a healthy relationship with social media, it is essential for individuals to establish boundaries, prioritize self-care, and be mindful of their online interactions. By being conscious of how social media affects their mental health, users can take proactive steps to safeguard their well-being. Seeking help from mental health professionals and support groups can also play a crucial role in mitigating the negative impact of social media on mental health. Remember, your mental health is important both online and offline.

<a name='1'></a>
 ## Other Popular Models as LLM for your Agents

#### Hugging Face (HuggingFaceHub endpoint)

```Python
from langchain_community.llms import HuggingFaceHub

llm = HuggingFaceHub(
    repo_id="HuggingFaceH4/zephyr-7b-beta",
    huggingfacehub_api_token="<HF_TOKEN_HERE>",
    task="text-generation",
)

### you will pass "llm" to your agent function
```

#### Mistral API

```Python
OPENAI_API_KEY=your-mistral-api-key
OPENAI_API_BASE=https://api.mistral.ai/v1
OPENAI_MODEL_NAME="mistral-small"
```

#### Cohere

```Python
from langchain_community.chat_models import ChatCohere
# Initialize language model
os.environ["COHERE_API_KEY"] = "your-cohere-api-key"
llm = ChatCohere()

### you will pass "llm" to your agent function
```

### For using Llama locally with Ollama and more, checkout the crewAI documentation on [Connecting to any LLM](https://docs.crewai.com/how-to/LLM-Connections/).